# *Description:*
    This project will convert YouTube videos into .wav files in one of two ways:
    1. Single item processing
        User will manually enter URLs as prompted through this notebook
    2. Bulk processing
        User will manually enter multiple URLs within the file "bulk_youtube_url_input.csv"

In [ ]:
#
# Dependencies
#

%pip install yt_dlp -q --disable-pip-version-check
%pip install ffmpeg -q --disable-pip-version-check
%pip install pydub -q --disable-pip-version-check

from __future__ import unicode_literals
from pydub import AudioSegment

import sys, os
import logging
import math
import csv
import ffmpeg
import yt_dlp # More information can be found here: https://github.com/yt-dlp/yt-dlp

# Logging set-up for further analysis
logging.basicConfig(
    filename='logs.log',
    filemode='w',
    format='[%(asctime)s][%(levelname)s] %(message)s',
    level=logging.DEBUG)
logging.info("Logger started")

In [ ]:
#
# Helper functions
#

def check_input(prompt, acceptable_responses):
    ''' Ensures that the field is an acceptable response '''
    while True:
        value = input(prompt)
        value = value.strip() # Removed redundant spaces
        value = value.lower()
        value = str(value)
        if value not in acceptable_responses:
            print('Sorry, this is not an acceptable response.')
            continue
        else:
            break
    return value

def clean_url(url):
    ''' Lightweight URL preprocessing '''
    url = url.strip()
    url = url.replace(',','')

    return url

### User Configurations
#### 1. YouTube Downloader options
#### 2. User will choose between single item processing and bulk processing

In [ ]:
# YouTube Downloader options
ydl_opts = {
    'format': 'bestaudio/best',
    'restrictfilenames': True,
    'outtmpl': os.getcwd()+'\\wav_downloads\\%(title)s', # specifies output location and formatting
    'force-overwrites': True,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav'
    }],
}
logging.info("YouTube downloader options:\n"+str(ydl_opts))

# Option 1: single item processing
# Option 2: bulk item processing
question = 'Would you like to...\n1. Download YouTube videos one URL at a time?\n2. Download multiple YouTube videos given an input file?\nAnswer: '

processing_type = check_input(question,{'1','2'})
processing_type_string = 'Single Item Processing' if '1' else 'Bulk Processing'
logging.info("Processing type chosen: "+processing_type_string)

In [ ]:
failed_urls = []

if processing_type == '1':
    request = True
    index = 0
    
    while request:
        url = input('\nPlease enter your URL: ')
        logging.info('Raw url #'+str(index)+': '+url)
        
        url = clean_url(url)
        logging.info('New url #'+str(index)+': '+url)
        
        # YouTube video downloader to .wav
        try:
            logging.info('Attempting YouTube download to .wav')
            ydl = yt_dlp.YoutubeDL(ydl_opts)
            ydl.download(url)
            logging.info('Success!')
        except Exception as e:
            logging.exception(str(e))
            failed_urls.append((index,url))

        index = index + 1
        one_more_url = check_input('Would you like to download another YouTube video? (yes or no)\n', {'yes','no'})
        if one_more_url == 'no':
            request = False
else:
    print('\nPlease insert your YouTube links within \"bulk_youtube_url_input.csv\"')
    
    file_name = 'bulk_youtube_url_input.csv'
    with open(file_name, newline='') as csvfile:
        input_file = csv.reader(csvfile, delimiter='\n')
        logging.info('Input file name: '+file_name)
                     
        next(input_file) # skips the header
        
        for index, value in enumerate(input_file):
            # Skip empty strings
            if not value:
                continue
            
            url = value[0]
            logging.info('Raw url #'+str(index)+': '+url)
            
            url = clean_url(url) # url preprocessing
            
            print('\nURL #'+str(index)+': '+url+'\n')
            logging.info('New url #'+str(index)+': '+url)
            
            # YouTube video downloader to .wav
            try:
                logging.info('Attempting YouTube download to .wav')
                ydl = yt_dlp.YoutubeDL(ydl_opts)
                ydl.download(url)
                logging.info('Success!')
            except Exception as e:
                logging.exception(str(e))
                failed_urls.append((index,url))

print('Failed URLs:')
print(failed_urls)